# ImagePredictionDataset Class Usage

This notebook contains examples for how the [ImagePredictionDataset](https://github.com/MIT-AI-Accelerator/multiearth-challenge/blob/c2318665ab94451eea4bd1b9e31a71655c6be001/src/multiearth_challenge/datasets/prediction_dataset.py#L11) class can be used to sample data from the NetCDF files provided as part of the MultiEarth challenge.

In [ ]:
import pkg_resources

from matplotlib import pyplot as plt
import numpy as np

from multiearth_challenge.datasets import prediction_dataset as pd

%matplotlib inline
%load_ext autoreload
%autoreload 2

### Specifying Dataset Data
In this example, Sentinel-2 visible imagery will serve as the source imagery and Landsat-8 imagery will serve as the target.

In [ ]:
# Set data paths to sample data included as part of the MultiEarth repository
source_files = [pkg_resources.resource_filename("multiearth_challenge", "data/sample_dataset/sent2_sample.nc")]
target_files = [pkg_resources.resource_filename("multiearth_challenge", "data/sample_dataset/landsat8_sample.nc")]

Specify the bands to include for the source and target data. This is set with a dictionary whose keys are the sensor and the values are a list of bands.</br>
Acceptable sensor and band values are:</br>
"Landsat-5": ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'ST_B6', 'SR_B7', 'QA_PIXEL']</br>
"Landsat-8": ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10', 'QA_PIXEL']</br>
"Sentinel-1": ['VV', 'VH']</br>
"Sentinel-2": ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'QA60']</br>
"Fire": ['BurnDate', 'ConfidenceLevel', 'LandCover', 'ObservedFlag']</br>
"Deforestation": None, There is only a single unnamed band associated with the deforestation data so there are no bands to select</br>

In [ ]:
source_bands = {
    "Sentinel-2": ["B4", "B3", "B2"], # RGB visible bands
}
target_bands = {
    "Landsat-8": ["SR_B4", "SR_B3", "SR_B2"], # RGB visible bands
}

### Creating the Dataset
Set additional parameters used by the ImagePredictionDataset

In [ ]:
# If True, returned images will have multiple channels in increasing order of frequency (e.g., red, green, blue for visible), co-pol before cross-pol, and with bands not originating from collected imagery coming last and in alphabetical order. 
# The metadata returned with the imagery will also specify the channel order. If False, each band is treated as a separate sample.
merge_source_bands = False # bool
merge_target_bands = False # bool

# The maximum allowable cloud coverage allowed in the source / target imagery as a fraction [0, 1]. Setting the maximum above 0 may be useful to incorporate additional samples even if the source image is slightly obscured.
# Note, there may be some innacuracies in the identified cloud coverage provided by the sensor's QA bands. This is especially true for Sentinel-2 data.
max_source_cloud_coverage = 0.0 # float
max_target_cloud_coverage = 0.0 # float

# The minimum and maximum inclusive relative time window in days around the target image from which source imagery is pulled. 
# If the minimum is None, there is no filter on the minimum relative date. Similarly, no maximum can be specified with a value of None.
# For example, a value of (None, -365) would result in only source imagery collected at least 365 days before the target image date being returned. 
source_date_window = (None, -365) # Tuple[Optional[float], Optional[float]]

# If True, if no source or target image remain after data filtering, raise a ValueError, otherwise this dataset will have length 0. 
error_on_empty = True # bool

Create the dataset. The sample data is small, but depending on the number of images contained in the NetCDF files, calculating the cloud coverage statistics may take several minutes.

In [ ]:
dataset = pd.ImagePredictionDataset(
    source_files,
    target_files,
    source_bands,
    target_bands,
    source_date_window,
    max_source_cloud_coverage,
    max_target_cloud_coverage,
    merge_source_bands,    
    merge_target_bands,
    error_on_empty,
)   

### Data Returned by the Dataset
The dataset serves as a sequence of samples. Each call to \_\_getitem__ will return a two element tuple where the second element holds a dictionary with a single target image and associated metadata. The first element holds a list of dictionaries where each one holds a source image along with associated metadata at the same location that satisfies the source_date_window.

Note, the returned images for this example are cloud free and collected >365 days before the target image. Cloud coverage is determined by information in the sensor's QA band, which may have inaccuracies.

In [ ]:
print(f"Number of dataset samples: {len(dataset)}")

# Get sample index 2
source_data, target_data = dataset[3]
print(f"Number of returned source samples with example target image: {len(source_data)}")
print(f"Source data key values returned: {source_data[0].keys()}")
print(f"Target data key values returned: {target_data.keys()}")

### Single channel vs. Multi-channel Imagery
The returned source images in the dataset above are single band and there are separate paired samples for each band. Below is an example where the bands have been merged into a 3-channel RGB image.

In [ ]:
# Make a second dataset with merged bands
merge_source_bands = True
merge_target_bands = True
dataset_merged_bands = pd.ImagePredictionDataset(
    source_files,
    target_files,
    source_bands,
    target_bands,
    source_date_window,
    max_source_cloud_coverage,
    max_target_cloud_coverage,
    merge_source_bands,    
    merge_target_bands,
    error_on_empty,
)   
# 1/3 target samples results in 1/3 of the number of samples compared to separate bands.
print(f"Number of merged band dataset samples: {len(dataset_merged_bands)}") 
# Get sample index 3
source_data_merged_bands, target_data_merged_bands = dataset_merged_bands[3]
print(f"Number of returned source samples with merged bands: {len(source_data_merged_bands)}")
print(f"Shape of example source image without merging bands: {source_data[0]['image'].shape}")
print(f"List of bands associated with the example single band source image: {source_data[0]['bands']}")
print(f"Shape of example source image with merged bands: {source_data_merged_bands[0]['image'].shape}")
print(f"List of bands associated with the example multi-band source image: {source_data_merged_bands[0]['bands']}") # This list corresponds to the channels in the image with the first band corresponding to channel index 0, the second channel index 1, etc.

### Image Characteristics
The collected images may require normalization for visualization. Here we perform a simple normalization without color balancing.</br>
Note, the difference in ground sample distance (GSD) between Sentinel imagery and Landsat imagery may require resizing or some other way to handle the difference in scale.</br>
Also, it should be noted that the filtering of cloudy images uses the QA band which may have inaccuracies resulting in some images with high pixel value cloud coverage obscuring land. Below is one such example. Additional filtering or masking of cloud coverage can be applied if needed.

In [ ]:
def normalize(img):
    img = img.astype(np.float64)
    img -= np.mean(img)
    img_std = np.std(img)
    img += img_std
    img /= img_std * 3.0
    img = np.clip(img, 0, 1)
    return img

In [ ]:
source_img_index = 0
fig1, axs1 = plt.subplots(1, 2)
fig1.suptitle("Single Band Example (source / target)")
axs1[0].set_title(f"{source_data[0]['data_source']}\n{source_data[source_img_index]['bands']}")
_ = axs1[0].imshow(normalize(source_data[source_img_index]["image"].squeeze()), cmap="gray")
axs1[1].set_title(f"{target_data['data_source']}\n{target_data['bands']}")
_ = axs1[1].imshow(normalize(target_data["image"].squeeze()), cmap="gray")

fig2, axs2 = plt.subplots(1, 2)
fig2.suptitle("Multi-Band Example (source / target)")
axs2[0].set_title(f"{source_data_merged_bands[source_img_index]['data_source']}\n{source_data_merged_bands[source_img_index]['bands']}")
_ = axs2[0].imshow(normalize(source_data_merged_bands[source_img_index]["image"]).transpose((1, 2, 0)), cmap="gray")
axs2[1].set_title(f"{target_data_merged_bands['data_source']}\n{target_data_merged_bands['bands']}")
_ = axs2[1].imshow(normalize(target_data_merged_bands["image"]).transpose((1, 2, 0)), cmap="gray")

### Closing the Datasets
The ImagePredictionDataset class holds open file handles to NetCDF files which need to be manually closed.

In [ ]:
dataset.close()
dataset_merged_bands.close()

### Incorporation into Training and Evaluation
This dataset can be wrapped in a straightforward manner for use in a desired ML training / evaluation framework, allowing for selection of desired data from within the returned dictionary, applying data transforms such as image resizing, and converting to framework compatible types.